In [1]:
print('Cloning the repo...')
!git clone https://github.com/beratcmn/cortec.git
%cd ./cortec/
print('\nAvailable files:')
%ls
print('\nDownloading the requirements...')
%pip install -qqq -U -r requirements.txt

Cloning the repo...
Cloning into 'cortec'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 126 (delta 20), reused 41 (delta 11), pack-reused 75
Receiving objects: 100% (126/126), 57.34 MiB | 29.05 MiB/s, done.
Resolving deltas: 100% (30/30), done.
Updating files: 100% (129/129), done.
/content/cortec

Available files:
CORTEC.ipynb  questions.csv  requirements.txt  results.md  TODO.md
data/         README.md      results.csv       src/

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.8/437.8 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [2]:
%pip show pybox

Name: pybox
Version: 0.0.3b1
Summary: A Python library for making common things easier and faster such as creating Chat UIs with Gradio or testing LLMs on predefined questions.
Home-page: https://github.com/beratcmn/pybox
Author: Berat Çimen
Author-email: beratcmn@hotmail.com
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: gradio, pandas
Required-by: 


In [62]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage
from langchain.schema.messages import SystemMessage
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.tools import BaseTool

from langchain.prompts import PromptTemplate

from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.agents import initialize_agent

from pybox.llm import GradioChatWebUI
from pybox.llm import Tester as LLMTester

from typing import Union

import json

In [4]:
import os
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["OPENAI_API_BASE"] = "https://demo-moh-opeanaij-ape.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "a275c1b36f42488eb96add2a955aba48"

In [5]:
model = AzureChatOpenAI(
    deployment_name="Demo-CORTEC-gpt-4",
    model_name="gpt-4",
    temperature=0
)

In [6]:
result = model(
    [
        HumanMessage(content="What are some touristic places in Paris?")
    ]
)

print(result.content)

1. Eiffel Tower: The most iconic symbol of Paris, offering panoramic views of the city.

2. Louvre Museum: The world's largest art museum and a historic monument in Paris.

3. Notre-Dame Cathedral: A masterpiece of French Gothic architecture.

4. Montmartre: Known for its bohemian past and the beautiful Sacré-Cœur Basilica.

5. Champs-Élysées: The most famous avenue in Paris, leading up to the Arc de Triomphe.

6. Palace of Versailles: A royal château in Versailles, known for its Hall of Mirrors and beautiful gardens.

7. Seine River: Offers boat tours that pass by many of Paris's most famous landmarks.

8. Musée d'Orsay: An art museum housed in a former railway station, known for its collection of Impressionist masterpieces.

9. Sainte-Chapelle: A stunning Gothic chapel known for its magnificent stained glass windows.

10. Latin Quarter: A historic and vibrant neighborhood known for its student life, as it is home to the Sorbonne University.

11. Centre Pompidou: A modern and contempo

In [7]:
embeddings = OpenAIEmbeddings(
    deployment="Demo-CORTEC-text-embedding-ada",
    model="text-embedding-ada-002",
    openai_api_base="https://demo-moh-opeanaij-ape.openai.azure.com/",
    openai_api_type="azure",
    chunk_size=16,
)

In [8]:
loader = DirectoryLoader('./data/CORTEC_cleaned/', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
texts = text_splitter.split_documents(documents)

print("Total texts:", len(texts))

sources = []
for doc in documents:
  doc.metadata["source"] = doc.metadata["source"][20:]
  sources.append(doc.metadata["source"])

print("Document sources has been formatted.")

sources = list(set(sources))
sources.sort()

print("Available sources:", sources)

Total texts: 3428
Document sources has been formatted.
Available sources: ['00 CorroLogic AST Sales Playbook-R0.pdf', '0002 Rev A Technical Report Crush Test of Drip Irrigation Tubing under Static Load.pdf', '0113 Rev 02 System Description.pdf', '651_Ed3_with_highlighted_changes_final.pdf', 'CHAPTER 62-762  ABOVEGROUND STORAGE TANK SYSTEMS.pdf', 'CorroLogic AST Project Reference List.pdf', 'Corrologic AST Solution for Storage Tanks at Bab Far North Field Project.pdf', 'Corrosion Inhibition of Stress Corrosion Cracking and Localized corrosion.pdf', 'Cortec CorroLogic system for ASTs.pdf', 'Cortec Corrologic technology for AST bottoms.pdf', 'Cortec Corrologic technology for protection of tank bottoms.pdf', 'Cortec VpCI technology for above ground storage tanks.pdf', 'Design, Construction, Operation,  Maintenance, and Inspection of  Terminal and Tank Facilities.pdf', 'Evaluation of Corrosion Protection Provided by VpCI-609 in Sand saturate....pdf', 'FAILURE  ANALYSIS OF  BOTTOM PLATE OF P

In [9]:
documents[0]

Document(page_content=' Evaluation of Impressed Current Cathodic ally Protected API1 650 Tank Bottoms in the Presence \nof Vapor Phase Corrosion Inhibitor  \n \n \nKhalil Abed  \nCortec Middle East FZC \nSheikh Zayed Road  \nDubai, P.O. Box 115133  \nUnited Arab Emirates  \n \nPankaj Pancha l  \nAbdulla Fouad -Impalloy Ltd. Co.  \n10th Street  \nDammam, Eastern Province, P.O. Box 257  \nKingdom of Saudi Arabia  \n Amish Gandhi  \nMetal Samples Company  \n152 Metal Samples Rd.  \nMunford, AL 36268 , P.O. Box 8   \nUnited States of America  \n \n \nABSTRACT  \n \nThis work aims to assess the effectiveness of amine carboxylate based Vapor Phase  Corrosion Inhibitor \n(VPCI) on the protection of storage tank bottoms against soil -side corrosion , as standalone and in \ncombination with Impressed Current  Cathodic Protection (ICCP) system.  It also attempts to determine \nthe effect of VPCI on instant -off potential.  Lab-scale tanks  simulating the environment of single bottom \nstorage ta

In [10]:
persist_directory = 'db'

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embeddings,
                                 persist_directory=persist_directory)

retriever = vectordb.as_retriever(search_kwargs={"k": 10})

In [53]:
_template = """
You are responsible for finding the documents I want. I will provide you with the all available documents and what I am searching.

Find me all the relevant documents.

If there is no matching document answer with an empty JSON object.

Answer without commentary and answer in JSON format.

Valid JSON Format:
  "documents": []

Input: {input}

Documents: {sources}
""".strip()

prompt = PromptTemplate(
    input_variables=["input", "sources"],
    template=_template,
)

document_find_chain = LLMChain(llm=model, prompt=prompt)

result = document_find_chain.run({
    "input": "C2020",
    "sources": sources
})

print(result)

{
  "documents": ["Paper_C2020-15065.pdf"]
}


In [81]:
def get_paper(user_input: str):
  global documents

  result = document_find_chain.run({
    "input": user_input,
    "sources": sources
  })

  result = json.loads(result)

  if len(result["documents"]) == 0:
    return "No document found."

  if len(result["documents"]) > 1:
    result["documents"] = [result["documents"][0]]

  docs = []
  for doc in documents:
    for paper_no in result["documents"]:
      if doc.metadata["source"] == paper_no:
        docs.append(doc)

  document = ""
  for doc in docs:
    document += doc.page_content

  return document

class PaperSearchTool(BaseTool):
    name = "Paper Search Tool"
    description = "This tool is useful when you need to retrieve documents with specific Paper Numbers."

    def _run(self, user_input: str):
        response = get_paper(user_input)
        return response

    def _arun(self, radius: Union[int, float]):  # async run
        raise NotImplementedError("This tool does not support async")

In [82]:
document_search_tool = create_retriever_tool(
    retriever,
    "search_cortec_documents",
    "Searches and returns documents regarding the CORTEC."
)

#paper_search_tool = create_retriever_tool(
#    retriever,
#    "search_cortec_papers",
#    "This tool is useful when you need to retrieve documents with specific Paper Numbers."
#)

paper_search_tool = PaperSearchTool()

tools = [document_search_tool, paper_search_tool]

In [83]:
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

_system_message = """
You are an AI assistant representing the company CORTEC. CORTEC is a company that specializes in corrosion protection solutions and offers a wide range of products and services for various industries around the globe.

Do your best to answer the questions. You are provided with a tool that has the company's files. Try to use the provided tool as much as possible.

If the user is asking for a specific document or a paper try to find the closest document with the relevant Paper No.
""".strip()

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=[],
    llm=model,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory,
)

new_prompt = agent.agent.create_prompt(
    system_message=_system_message,
    tools=tools
)

agent.agent.llm_chain.prompt = new_prompt
agent.tools = tools

agent

AgentExecutor(memory=ConversationBufferWindowMemory(chat_memory=ChatMessageHistory(messages=[]), output_key=None, input_key=None, return_messages=True, human_prefix='Human', ai_prefix='AI', memory_key='chat_history', k=5), callbacks=None, callback_manager=None, verbose=True, tags=['chat-conversational-react-description'], metadata=None, agent=ConversationalChatAgent(llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=ChatPromptTemplate(input_variables=['input', 'chat_history', 'agent_scratchpad'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="You are an AI assistant representing the company CORTEC. CORTEC is a company that specializes in corrosion protection solutions and offers a wide range of products and services for various industries around the globe.\n\nDo your best to answer the question

In [94]:
print(agent.agent.llm_chain.prompt.messages[2].prompt.template)

TOOLS
------
Assistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:

> search_cortec_documents: Searches and returns documents regarding the CORTEC.
> Paper Search Tool: This tool is useful when you need to retrieve documents with specific Paper Numbers.

RESPONSE FORMAT INSTRUCTIONS
----------------------------

When responding to me, please output a response in one of two formats:

**Option 1:**
Use this if you want the human to use a tool.
Markdown code snippet formatted in the following schema:

```json
{{
    "action": string, \ The action to take. Must be one of search_cortec_documents, Paper Search Tool
    "action_input": string \ The input to the action
}}
```

**Option #2:**
Use this if you want to respond directly to the human. Markdown code snippet formatted in the following schema:

```json
{{
    "action": "Final Answer",
    "action_input": string \ You should put what 

In [75]:
agent("Hello")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Hello! How can I assist you with your corrosion protection needs today?"
}
```

> Finished chain.


{'input': 'Hello',
 'chat_history': [],
 'output': 'Hello! How can I assist you with your corrosion protection needs today?'}

In [76]:
agent("Can you summarize Paper No C2020")["output"]



> Entering new AgentExecutor chain...
```json
{
    "action": "Paper Search Tool",
    "action_input": "C2020"
}
```
Observation: C
omparison of VCI and CP Performance Using Floor Scan Data from  Several 10-Year Old 
AST Floors  
Tim Whited 
MESA  
4445 S. 74th E. Ave.  
Tulsa, Oklahoma 74145 
Tim.Whited@mesaproducts.com  
ABS
TRACT  
An especially noteworthy and important analysis will be provided on the real-time field 
performance of stand-alone vapor corrosion inhibitor (VCI) systems in comparison to cathodic 
protection (CP) systems on aboveground storage tank (AST) floors . API 653 floor inspection data 
will be presented on a group of (12) AST’s that all had completely  new steel floors installed, then 
were inspected (7-10) years later with magnetic flux leakage ( MFL) and ultrasonic testing (UT ).  
VCI was used to control soil -side corrosion on approximately 42% of the tanks and CP was used 
on the other 58%.  The tanks  resided inside pipeline stations located within 200-

"The paper titled 'Comparison of VCI and CP Performance Using Floor Scan Data from Several 10-Year Old AST Floors' (Paper No. C2020) by Tim Whited provides a comparative analysis of the performance of Vapor Corrosion Inhibitor (VCI) systems and Cathodic Protection (CP) systems on aboveground storage tank (AST) floors. The paper presents inspection data from 12 ASTs that had new steel floors installed and were inspected 7-10 years later. VCI was used to control soil-side corrosion on approximately 42% of the tanks and CP was used on the remaining 58%. The paper concludes that both VCI and CP are effective corrosion control systems, with VCI providing a very viable soil-side corrosion solution for AST floors. The corrosion mitigation on the VCI only tanks was found to be as effective as the tanks with CP only where the CP was operationally sufficient."

In [77]:
agent("please summarize paper 11030")["output"]



> Entering new AgentExecutor chain...
```json
{
    "action": "Paper Search Tool",
    "action_input": "11030"
}
```
Observation:     A study on the Effect of Volatile Corrosion Inhibitors on Impressed Current Cathodic 
Protection  
 
Khalil M. Abed  
CORTEC Middle -East 
Sheikh Zayed Road  
Dubai, P.O. Box 115133  
United Arab Emirates  
 
Calvin R. Pynn  
AEGIS Technical Systems LLC  
26th S treet - Al Quoz 4  
Dubai, P.O. Box 37894  
United Arab Emirates  
 
 
 
ABSTRACT  
 
The physical and electrolytic contact between the bottom plate of an aboveground storage tank and the 
underlying soil typically varies over the area of a tank bottom plates. Externa l tank bottom plates are 
exposed to both electrolytic and vapor -phase corrosive environments. Cathodic Protection (CP) 
requires direct electrolytic contact between the tank floor and the underlying soil to effectively mitigate 
corrosion; hence it is ineffe ctive in a vapor -phase environment.  
 
There is a growing trend to su

"The paper titled 'A study on the Effect of Volatile Corrosion Inhibitors on Impressed Current Cathodic Protection' (Paper No. 11030) by Khalil M. Abed and Calvin R. Pynn investigates the compatibility and interactions of three different volatile corrosion inhibitors and cathodic protection when applied jointly on an oxygen concentration corrosion macro-cell setup. The results varied significantly between the three volatile corrosion inhibitors. One showed a cathodic polarization effect and resulted in a reduction of CP current requirement by 48%. Another had an anodic polarization effect and resulted in a reduction of CP current requirement by 2%. The third had no polarization effect, and resulted in an increase of CP current requirement by 10%. The paper concludes that the electrochemical interaction characteristics of a volatile corrosion inhibitor must be evaluated and verified before it is applied in conjunction with cathodic protection to both avoid risk of incompatibility and ma

In [78]:
agent("Can you summarize Paper No C2023")["output"]



> Entering new AgentExecutor chain...
```json
{
    "action": "Paper Search Tool",
    "action_input": "C2023"
}
```
Observation: No document found.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "I'm sorry, but I couldn't find a document corresponding to Paper No. C2023 in the database."
}
```

> Finished chain.


"I'm sorry, but I couldn't find a document corresponding to Paper No. C2023 in the database."

In [79]:
agent("what is Compatibility & Interactions between Cathodic Protection and a Vapor?")["output"]



> Entering new AgentExecutor chain...


```json
{
    "action": "search_cortec_documents",
    "action_input": "Compatibility & Interactions between Cathodic Protection and a Vapor"
}
```
Observation: [Document(page_content='Compatibility & Interactions between Cathodic Protection and a Vapor \nPhase Corrosion Inhibitor \n \nCalvin R. Pynn  \nAEGIS Technical Systems LLC  \nDubai, United Arab Emirates  \nEmail: Calvin.Pynn@eim.ae  \n \nKhalil Abed  \nCORTEC Middle -East \nDubai, United Arab Emirates  \nEmail: kabed@cortec -me.com  \n \n \nABSTRACT\n \nThe nature of the physical contact between the bottom plate of an above ground storage tank and the', metadata={'page': 0, 'source': 'data/CORTEC_cleaned/Paper_9232.pdf'}), Document(page_content='Compatibility & Interactions between Cathodic Protection and a Vapor \nPhase Corrosion Inhibitor \n \nCalvin R. Pynn  \nAEGIS Technical Systems LLC  \nDubai, United Arab Emirates  \nEmail: Calvin.Pynn@eim.ae  \n \nKhalil Abed  \nCORTEC Middle -East \nDubai, United Arab Emirates  \nEmail

'The compatibility and interactions between Cathodic Protection (CP) and a Vapor Phase Corrosion Inhibitor (VCI) are complex and can vary depending on the specific VCI used. CP is a proven corrosion mitigation technique where the target metal surface is in contact with a conductive electrolyte such as moist soil, but it is not effective in void spaces where there is no electrolyte contact with the tank floor. On the other hand, VCIs are increasingly being used to ensure corrosion is mitigated in these void spaces. However, the electrochemical interaction characteristics of a VCI must be evaluated and verified before it is applied in conjunction with CP to both avoid risk of incompatibility and maximize the benefit of compatibility.'

In [84]:
agent("please summarize paper 9232")["output"]



> Entering new AgentExecutor chain...
```json
{
    "action": "Paper Search Tool",
    "action_input": "9232"
}
```
Observation:   Compatibility & Interactions between Cathodic Protection and a Vapor 
Phase Corrosion Inhibitor 
 
Calvin R. Pynn  
AEGIS Technical Systems LLC  
Dubai, United Arab Emirates  
Email: Calvin.Pynn@eim.ae  
 
Khalil Abed  
CORTEC Middle -East 
Dubai, United Arab Emirates  
Email: kabed@cortec -me.com  
 
 
ABSTRACT
 
The nature of the physical contact between the bottom plate of an above ground storage tank and the 
underlying foundation/soil varies considerably over the area of a tank bottom  and from tank to tank . 
This may vary from  direct electrolytic contact between the plate and moist underlying soil to void  
spaces  between the plate and the soil due to floor buckling and/or soil settlement.  
Cathodic protection is a  proven corrosion mitigation technique where the target metal surf ace is in 
contact with a conductive electrolyte such as moist so

"The paper titled 'Compatibility & Interactions between Cathodic Protection and a Vapor Phase Corrosion Inhibitor' (Paper No. 9232) discusses the compatibility and interactions between cathodic protection and a vapor phase corrosion inhibitor in a liquid-phase environment. The study involved laboratory experiments to evaluate the effects of a specific amine carboxylate-based inhibitor and cathodic protection, when applied individually and jointly on an oxygen concentration corrosion macro-cell in a salt water solution. The results indicated a beneficial synergistic effect between the inhibitor tested and cathodic protection. The inhibitor enhances cathodic polarization to reduce cathodic protection current requirement, and the cathodic protection reduction reaction appears to enhance the absorption & effectiveness of the inhibitor at the cathodic metal surface."

In [98]:
def get_message(question:str):
  return agent(question)["output"]


tester = LLMTester(bot_callback=get_message, questions_csv_path="./questions.csv", questions=[])

tester.start()



> Entering new AgentExecutor chain...
```json
{
    "action": "Paper Search Tool",
    "action_input": "15065"
}
```
Observation: C
omparison of VCI and CP Performance Using Floor Scan Data from  Several 10-Year Old 
AST Floors  
Tim Whited 
MESA  
4445 S. 74th E. Ave.  
Tulsa, Oklahoma 74145 
Tim.Whited@mesaproducts.com  
ABS
TRACT  
An especially noteworthy and important analysis will be provided on the real-time field 
performance of stand-alone vapor corrosion inhibitor (VCI) systems in comparison to cathodic 
protection (CP) systems on aboveground storage tank (AST) floors . API 653 floor inspection data 
will be presented on a group of (12) AST’s that all had completely  new steel floors installed, then 
were inspected (7-10) years later with magnetic flux leakage ( MFL) and ultrasonic testing (UT ).  
VCI was used to control soil -side corrosion on approximately 42% of the tanks and CP was used 
on the other 58%.  The tanks  resided inside pipeline stations located within 200-